In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import numpy as np
import pickle

In [4]:
# Define the convolutional part of the architecture
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=6, out_channels=32, kernel_size=5, padding=2)
        self.pool1 = nn.MaxPool1d(kernel_size=2)
        self.conv2 = nn.Conv1d(
            in_channels=32, out_channels=64, kernel_size=3, padding=1
        )
        self.pool2 = nn.MaxPool1d(kernel_size=2)
        self.conv3 = nn.Conv1d(
            in_channels=64, out_channels=128, kernel_size=3, padding=1
        )

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool1(x)
        x = F.relu(self.conv2(x))
        x = self.pool2(x)
        x = F.relu(self.conv3(x))
        return x


# Define the recurrent part of the architecture
class RecurrentNet(nn.Module):
    def __init__(self):
        super(RecurrentNet, self).__init__()
        self.lstm = nn.LSTM(input_size=128, hidden_size=128, batch_first=True)
        self.fc = nn.Linear(128, 1)

    def forward(self, x):
        self.lstm.flatten_parameters()
        x, _ = self.lstm(x)
        x = torch.sigmoid(self.fc(x))
        return x


# Combine convolutional and recurrent parts into one model
class EndToEndModel(nn.Module):
    def __init__(self):
        super(EndToEndModel, self).__init__()
        self.conv_net = ConvNet()
        self.recurrent_net = RecurrentNet()

    def forward(self, x):
        x = self.conv_net(x)
        x = x.permute(
            0, 2, 1
        )  # Prepare for LSTM: (batch_size, sequence_length, features)
        x = self.recurrent_net(x)
        return x

In [16]:
# Pre-processing

In [17]:
# Augmentation

In [ ]:
# Load .pkl data
def load_pkl_data(X_path, Y_path):
    with open(X_path, "rb") as f:
        X = pickle.load(f)  # List of numpy arrays
    with open(Y_path, "rb") as f:
        Y = pickle.load(f)  # List of numpy arrays
    return X, Y


# Dataset class
class IMUDataset(Dataset):
    def __init__(self, X, Y, sequence_length=128, downsample_factor=4):
        self.data = []
        self.labels = []
        self.sequence_length = sequence_length
        self.downsample_factor = downsample_factor

        # Process each session
        for imu_data, labels in zip(X, Y):
            imu_data = self.normalize(imu_data)  # Normalize data
            num_samples = len(labels)
            for i in range(0, num_samples, sequence_length):
                imu_segment = imu_data[i : i + sequence_length]
                label_segment = labels[i : i + sequence_length]

                # Ensure segments have the correct length
                if len(imu_segment) == sequence_length:
                    self.data.append(imu_segment)
                    # Downsample labels to match reduced sequence length
                    downsampled_labels = label_segment[:: self.downsample_factor]
                    self.labels.append(downsampled_labels)

    def normalize(self, data):
        # Normalize each channel independently
        mean = np.mean(data, axis=0)
        std = np.std(data, axis=0)
        return (data - mean) / (std + 1e-5)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x = self.data[idx]
        y = self.labels[idx]
        x = torch.tensor(x, dtype=torch.float32)
        y = torch.tensor(y, dtype=torch.float32)
        return x, y


# Paths to .pkl files
X_path = "./dataverse_files/pkl_data/pkl_data/DX_I_X.pkl"
Y_path = "./dataverse_files/pkl_data/pkl_data/DX_I_Y.pkl"

# Load data
X, Y = load_pkl_data(X_path, Y_path)

# Prepare dataset and dataloader
dataset = IMUDataset(X, Y)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Example: Iterate through the dataloader
for batch_x, batch_y in dataloader:
    print("Batch X shape:", batch_x.shape)
    print("Batch Y shape:", batch_y.shape)
    break

Batch X shape: torch.Size([32, 128, 6])
Batch Y shape: torch.Size([32, 128])


In [7]:
# Training setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Hyperparameters
learning_rate = 1e-3
batch_size = 128
num_epochs = 20  # Adjust as needed
sequence_length = 128

# Update DataLoader with batch_size 128
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# Initialize the model, loss function, and optimizer
model = EndToEndModel().to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss for binary classification
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

Using device: cuda


In [ ]:
# Training loop
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_loss = 0.0

    for batch_x, batch_y in dataloader:
        # Prepare data
        batch_x = batch_x.permute(0, 2, 1).to(
            device
        )  # (batch_size, 6, sequence_length)
        batch_y = batch_y.to(device)  # (batch_size, sequence_length)

        # Forward pass
        outputs = model(batch_x)
        outputs = outputs.squeeze(
            -1
        )  # Remove the last dimension (batch_size, sequence_length)

        # Calculate loss
        loss = criterion(outputs, batch_y)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Accumulate loss
        running_loss += loss.item()

    # Print epoch statistics
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}")

# Save the trained model
torch.save(model.state_dict(), "end_to_end_model.pth")
print("Model training complete and saved as 'end_to_end_model.pth'.")

Using device: cuda
Epoch [1/20], Loss: 0.1024
Epoch [2/20], Loss: 0.0722
Epoch [3/20], Loss: 0.0649
Epoch [4/20], Loss: 0.0600
Epoch [5/20], Loss: 0.0554
Epoch [6/20], Loss: 0.0522
Epoch [7/20], Loss: 0.0494
Epoch [8/20], Loss: 0.0480
Epoch [9/20], Loss: 0.0462
Epoch [10/20], Loss: 0.0458
Epoch [11/20], Loss: 0.0447
Epoch [12/20], Loss: 0.0440
Epoch [13/20], Loss: 0.0435
Epoch [14/20], Loss: 0.0426
Epoch [15/20], Loss: 0.0418
Epoch [16/20], Loss: 0.0416
Epoch [17/20], Loss: 0.0410
Epoch [18/20], Loss: 0.0403
Epoch [19/20], Loss: 0.0400
Epoch [20/20], Loss: 0.0394
Model training complete and saved as 'end_to_end_model.pth'.
